In [40]:
# from Modules.LSH import semantic_query_lsh
# from Modules.LSH import LSH


# import numpy as np
# import os

In [41]:
# file_path = "./random_data.txt"
# read_data = np.loadtxt(file_path)
# plane_norms = LSH(read_data, 8)
# query=[read_data[0]]
# folder_name = "bucket_files"
# result = semantic_query_lsh(query, plane_norms,folder_name)

In [42]:
%load_ext autoreload
%autoreload 2


from utils import *
from Modules.LSH import*


datafile_path="../DataBase/random_data_1000.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:

# Test LSH_index
# Read Data From File
data_path = datafile_path
read_data = np.loadtxt(data_path)


# Layer(1)
# Convert to Dictionary TODO Take From Zeyad
level_1_in=array_to_dictionary(read_data)

Level_1_path = "../DataBase/Level1"

if os.path.exists(Level_1_path):
    shutil.rmtree(Level_1_path, ignore_errors=True, onerror=lambda func, path, exc: None)
os.makedirs(Level_1_path)


level_1_planes=LSH_index(data=level_1_in, nbits=8,index_path=Level_1_path)

In [44]:
# Layer(2)
# OnEach Bucket Apply LSH

# List all files in the directory
files = os.listdir(Level_1_path)

Level_2_path = "../DataBase/Level2"
if os.path.exists(Level_2_path):
    shutil.rmtree(Level_2_path, ignore_errors=True, onerror=lambda func, path, exc: None)
os.makedirs(Level_2_path)

level_2_planes={}
# Loop over the files
for file_name in files:
    file_path = os.path.join(Level_1_path, file_name)
    if os.path.isfile(file_path):
        # TODO Zeyad
        # Read Data
        read_data_2 = np.array(np.loadtxt(file_path,dtype=int)).reshape((-1,1))
        # no_of_elements=1 if not read_data.shape else read_data.shape[0]
        # # break
        vectors=np.array([[]])
    

        for id in read_data_2:
            vector=get_vector_from_id(datafile_path,id)
            # If vectors is still an empty array, initialize it with the first vector
            if vectors.size == 0:
                vectors = vector.reshape(1, -1)
            else:
                vectors = np.insert(vectors, len(vectors), [vector], axis=0)
        
        level_2_in=array_to_dictionary(values=vectors,keys=np.hstack(read_data_2))

        # # Apply LSH on this Bucket
        level_2_planes[file_name[:-4]]=LSH_index(data=level_2_in, nbits=8,index_path='../DataBase/Level2/'+file_name[:-4])



In [46]:
# Query
query=read_data[18]
# print(query)
# Layer (1)
bucket = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path="../DataBase/Level1")
print(bucket)

# Layer(2)
bucket = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket],index_path="../DataBase/Level2/"+bucket)
print(bucket)


[ 18 248]
01110101
18
11000001


In [47]:
# Query
query=read_data[304]
# print(query)
# Layer (1)
bucket = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path="../DataBase/Level1")
print(bucket)

# Layer(2)
bucket = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket],index_path="../DataBase/Level2/"+bucket)
print(bucket)

[304 422 766 921]
00000100
304
10100110
